<a href="https://colab.research.google.com/github/SrishaRavi-SrishaRavi/RAG-Pipeline-using-Wikipedia---Distilbert/blob/main/Building_RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets import all the libraries required

## **## Step 1: Retrieving Knowledge from Wikipedia**

In [2]:
import wikipedia
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def get_wikipedia_content(topic):
  try:
    page = wikipedia.page(topic)
    return page.content
  except wikipedia.exceptions.PageError:
    return None
  except wikipedia.exceptions.DisambiguationError as e:
    #handle cases where the topic is ambigious
    print(f"Ambiguous topic, please be more specific. Options:{e.options}")
    return None

topic = input("Enter a topic you want to learn about:")
document = get_wikipedia_content(topic)

if not document:
  print("Sorry, I am not able to retrieve the particular document, please check your spelling once")
  exit()

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
def split_text(text,chunk_size=256,chunk_overlap =20):
  tokens = tokenizer.tokenize(text)
  chunks = []
  start =0
  while start < len(tokens):
    end = min(start + chunk_size,len(tokens))
    chunks.append(tokenizer.convert_tokens_to_string(tokens[start:end]))
    if end ==len(tokens):
      break
    start = end - chunk_overlap
  return chunks

chunks = split_text(document)
print(f"Number of chunks:{len(chunks)}")

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = embedding_model.encode(chunks)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


query = input("Ask a question about this topic")
query_embeddings = embedding_model.encode([query])

k=3
distances,indices = index.search(np.array(query_embeddings),k)
retrieved_chunks = [chunks[i] for i in indices[0]]
print("Retrieved Chunks")
for chunk in retrieved_chunks:
  print("- "+ chunk)


qa_model_name = "distilbert-base-uncased-distilled-squad"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering",model=qa_model,tokenizer=qa_tokenizer)

context = " ".join(retrieved_chunks)
answer = qa_pipeline(question=query,context=context)
print(f"Answer:{answer['answer']}")

Enter a topic you want to learn about:Indian mango fruit


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Device set to use cpu


Answer:sweet smell


Retrieving Wikipedia content based on user provided topic using the Wikipedia API.If the topic is valid, the function returns the page content; otherwise, it handles error by either notifiying the user a ambiuous topic with multiple options or exiting if no relevant page found.

In [3]:
!apt install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Wikipedia articles are long, splitting them into smaller overlapping chunks for better retrievel

Tokenizing the retrieved content from wikipedia and splitting it up into small overlapping chunks for efficient retrieval.
Pre-trainedTokenizer = all-mpnet-base-v2
chunk_szie = 256
overlap tokens= 20


## **Step 2: Storing and Retrieving Knowledge**

Converting text chunks into numerical embeddings using the Sentence Transformer model (all-mpnet-base-v2), which captures their semantic meaning.
Creating a FAISS index with an L2 (Euclidean) distance metric and store
the embeddings in it. This will allow us to efficiently retrieve the most relevant chunks based on a user’s query.

## **Step 3: Querying the RAG Pipeline**

*  Convert the query into an embedding.
*  Retrieve the top-k most relevant chunks using FAISS.
*  Use an LLM-powered question-answering model to generate the answer.


## **Step 4: Answering the Question with an LLM**